<a href="https://colab.research.google.com/github/christievanderuit/TM10007_group11/blob/main/TM10007_Group11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test

Hoi ik heb dit deel aangepast nadat ik hem denk ik heb afgesplitst in een eigen branch?
nu weer proberen te mergen met main branch

hoi hoi


In [ ]:
!pip install sklearn numpy matplotlib

In [1]:
# Run this to use from colab environment
!git clone https://github.com/jveenland/tm10007_ml.git
import os
import pandas as pd
data = pd.read_csv('/content/tm10007_ml/worcgist/GIST_radiomicFeatures.csv', index_col=0)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

Cloning into 'tm10007_ml'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 83 (delta 13), reused 12 (delta 12), pack-reused 64
Unpacking objects: 100% (83/83), 67.93 MiB | 15.16 MiB/s, done.
The number of samples: 246
The number of columns: 494


In [11]:
#zorgen dat verschillende functies uit de files aangeroepen kunnen worden 
from tm10007_ml.worcgist.load_data import *   #misschien dat dit nog op een nettere manier kan 
dataset = load_data(); 

In [29]:
#Splitsen in een train en in een testset 

# Classifiers
from sklearn.model_selection import train_test_split

#Waarbij x = features, y = label
#vraag: doet ie dit elke keer op een andere manier? want dan toch lastig conclusies te trekken? --> daarom achter random state een getal - maakt niet uit zolang integer is
y=data['label']
x = data.drop("label", axis='columns')
# print(y)
# print(x)
df_train, df_test = train_test_split(data, test_size=0.2, random_state=1)
print(df_train.shape, df_test.shape)

(196, 494) (50, 494)
